# Exercise 5.3

First of all, we import usefull libraries.

In [1]:
import numpy as np
import pandas as pd
from sklearn.base import clone
from sklearn.metrics import accuracy_score
from mlxtend.classifier import StackingClassifier
from itertools import combinations

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

Now, we import stored data.

In [2]:
%store -r data_set
%store -r labels
%store -r test_data_set
%store -r test_labels
%store -r unique_labels

We define a function which builds all classifiers we want to compare.

In [3]:
def build_classifiers():
    
    linear_regression = LinearRegression()
    linear_regression.fit(data_set, labels)
    
    neighbors = KNeighborsClassifier()
    neighbors.fit(data_set, labels)
    
    svc = SVC()
    svc.fit(data_set, labels)
    
    decision_tree_classifier = DecisionTreeClassifier()
    decision_tree_classifier.fit(data_set, labels)
    
    gaussian_nb = GaussianNB()
    gaussian_nb.fit(data_set, labels)
    
    qda = QuadraticDiscriminantAnalysis()
    qda.fit(data_set, labels)
    
    return linear_regression, neighbors, svc, decision_tree_classifier, gaussian_nb, qda

Then, we define another function to compare all of them.

In [4]:
def build_stacked_classifier(classifiers, stacked_classifier):
    output = []
    for classifier in classifiers:
        output.append(classifier.predict(data_set))
    output = np.array(output).reshape((130,3))
    
    # stacked classifier part:
    stacked_classifier.fit(output.reshape((130,3)), labels.reshape((130,)))
    test_set = []
    for classifier in classifiers:
        test_set.append(classifier.predict(test_data_set))
    test_set = np.array(test_set).reshape((len(test_set[0]),3))
    predicted = stacked_classifier.predict(test_set)
    return predicted

Finally, we build it and print their accuracy, so we're getting the best three ones.

In [5]:
all_classifiers = build_classifiers()
all_combinations = list(combinations(all_classifiers, 3))

_exit_ = False # To get out of the loops if we reach an accuracy of 1
accuracy = accuracy_old = 0.

for i in range(len(all_combinations)):
    classifiers = list(all_combinations[i])
    
    for j in range(6):
        if all_classifiers[j] not in classifiers: # stacked_classifier cannot be in classifiers
            stacked_classifier = clone(all_classifiers[j])
            predicted = build_stacked_classifier(tuple(classifiers), stacked_classifier).astype(int)
            accuracy = accuracy_score(test_labels, predicted)      
        
        if accuracy > accuracy_old:
            accuracy_old = accuracy
            final_classifiers = classifiers
            final_stacked_classifier = stacked_classifier
        
            if accuracy == 1.:
                _exit_ = True
                break   
    if _exit_:
        break

print('Best accuracy is:', accuracy_old, 'with classifiers: \n', final_classifiers, '\n The stacked classifier was:', final_stacked_classifier)

Best accuracy is: 1.0 with classifiers: 
 [LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False), KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'), SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)] 
 The stacked classifier was: GaussianNB(priors=None, var_smoothing=1e-09)


/Users/Manuela/anaconda2/envs/py3/lib/python3.5/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
